<a href="https://colab.research.google.com/github/vipashaaV321/Collaborative-Food-Recipe-Recommendation-System/blob/Sarvesh/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Drive and Importing Libraries

In [ ]:
Base_path ='/content/drive/MyDrive/RS Data/Assignment 2'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# Reading data files

In [ ]:
raw_recipies = pd.read_csv(Base_path+"/RAW_recipes.csv")
int_train = pd.read_csv(Base_path+"/interactions_train.csv")
int_val = pd.read_csv(Base_path+"/interactions_validation.csv")
int_test = pd.read_csv(Base_path+"/interactions_test.csv")

# Creating Useful database for Recommendation System

In [ ]:
raw_recipies.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
raw_recipies.shape

(231637, 12)

In [ ]:
# This will be used for application interface of the recommendation system to provide detailed information for user.
raw_recipies =raw_recipies[["name", "id","minutes","tags","steps","ingredients"]] 


In [ ]:
# Extracting name id dictionary and id name dictionary for further use 

# Name : Id Dictionary
name_id_dct = zip(raw_recipies.name,raw_recipies.id)
name_id_dct = dict(name_id_dct)

# Id : Name Dictionary 
id_name_dct = zip(raw_recipies.id,raw_recipies.name)
id_name_dct = dict(id_name_dct)

In [ ]:
len(id_name_dct)

231637

In [ ]:
int_train = int_train[["user_id","recipe_id","rating"]]
int_train.head()

,user_id,recipe_id,rating
0,2046,4684,5.0
1,2046,517,5.0
2,1773,7435,5.0
3,1773,278,4.0
4,2046,3431,5.0


In [ ]:
# Creating User - Item Rating Matrix 
rec_ids =  raw_recipies['id'].unique()
df =pd.DataFrame(columns= rec_ids)
df["user_id"] = 0 

In [ ]:
from scipy.sparse import csr_matrix
users = sorted(int_train['user_id'].unique())
items = sorted(int_train['recipe_id'].unique())

# Create a mapping from user and item ids to matrix indices
user_to_idx = {user: idx for idx, user in enumerate(users)}
item_to_idx = {item: idx for idx, item in enumerate(items)}

# Create a sparse matrix of user-item int_train
data = int_train['rating'].tolist()
row_indices = int_train['user_id'].apply(lambda x: user_to_idx[x]).tolist()
col_indices = int_train['recipe_id'].apply(lambda x: item_to_idx[x]).tolist()
user_item_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(len(users), len(items)))
user_item_df = pd.DataFrame(user_item_matrix.todense(), index=users, columns=items)

#User Item Matrix for ratings and Items in training set 
user_item_df

,38,40,45,46,49,52,53,58,59,62,...,536401,536411,536643,536679,536726,536729,536990,537175,537319,537458
1533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002204415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002214643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002227190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002254807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# This part will be omitted to my personal file 

#user_item_df1 = user_item_df.reindex(columns=user_item_df.columns.tolist() +["s"]).fillna(0)

# Ram Crashing for this approach or else the whole matrix would be ready to use.

In [ ]:
# user_item_df.to_csv("user_item_ratings.csv")